In [1]:

%load_ext autoreload
%autoreload 2
import utils
import config
from keywords_postprocess import deduplicate_keywords
import pandas as pd
import plotly.express as px
# px.bar(keywords.groupby("type")['type'].count(), text_auto=True).update_layout(yaxis_title="Count", xaxis_title="Type")

In [12]:
extracted = config.Keywords.load_extracted()
grants = config.Grants.load()
keywords = config.Keywords.load()

In [16]:
grants.explode("linked_organizations").groupby("linked_organizations")['id'].count().sort_values(ascending=False).head(20).to_clipboard()